## 1a. Calculate <i>P<sup>v</sup></i><sub>sat</sub> for a single melt composition from a dataframe using default options

## Setting things up
First we need to import a few Python packages (including VolFe) and set the precision of the calculations

In [1]:
# import python packages
import pandas as pd
import gmpy2 as gp
import VolFe as vf

# set calculation precision
gp.get_context().precision=50

At a minimum to run a Pvsat calculations, we need a dataframe of the melt composition, an estimate of oxygen fugacity and Fe in the melt [note 1], and temperature:

- Sample is just the name for this analysis.

- Temperature is in 'C.

- Volatile-free melt composition is in wt% oxides [note 2]. All these oxides must be present in the dataframe, so set them to 0. if you have no data for them (although see [note 1] around FeOT).

- H2O is all hydrogen in the melt reported as H2O in wt% [note 3]. 

- CO2ppm is all carbon in the melt reported as CO2 in ppm [note 3].

- STppm is all sulfur in the melt reported as S in ppm [note 3].

- Xppm is all "X" in the melt reported as X in ppm [note 3]. X is an unreactive melt species whose identity can be changed - this is explored WHERE.

- Fe3+FeT is the ratio of Fe3+ to FeT in the melt.

[note 1] In this example we specify oxygen fugacity using Fe3+/FeT and Fe in the melt as FeOT as is quite common for melt inclusion and matrix glass analyses - other options are possible, shown later in the notebook.

[note 2] It does not matter what the non-volatile oxides sum too - they are renormalised to 100 wt% minus the total of the volatiles (i.e., H2O + CO2 + ST + X).

[note 3] The volatile concentrations are absolute for the melt = i.e., the non-volatile melt composition is normalised to 100 wt% minus the volatiles. The volatiles are not added to the oxides and then renormalised.

The following composition is analysis TN273-01D-01-01 from Brounce et al. (2014), with a temperature chosen as 1200 'C.

M. N. Brounce, K. A. Kelley, E. Cottrell, Variations in Fe3+/∑Fe of Mariana Arc Basalts and Mantle Wedge fO2, Journal of Petrology, Volume 55, Issue 12, December 2014, Pages 2513–2536, https://doi.org/10.1093/petrology/egu065

In [2]:
# Define the melt composition, fO2 estimate, and T as a dictionary.
my_analysis = {'Sample':'TN273-01D-01-01',
           'T_C': 1200., # Temperature in 'C
           'SiO2': 56.98, # wt%
           'TiO2': 1.66, # wt%
           'Al2O3': 15.52, # wt%
           'FeOT': 9.47, # wt%
           'MnO': 0.24, # wt%
           'MgO': 2.96, # wt%
           'CaO': 6.49, # wt%
           'Na2O': 4.06, # wt%
           'K2O': 0.38, # wt%
           'P2O5': 0.22, # wt%
           'H2O': 1.88, # wt%
           'CO2ppm': 13., # ppm
           'STppm': 362.83, # ppm
           'Xppm': 0., # ppm
           'Fe3FeT': 0.171}

# Turn the dictionary into a pandas dataframe, setting the index to 0.
my_analysis = pd.DataFrame(my_analysis, index=[0])

# Show the DataFrame.
print(my_analysis)

            Sample     T_C   SiO2  TiO2  Al2O3  FeOT   MnO   MgO   CaO  Na2O  \
0  TN273-01D-01-01  1200.0  56.98  1.66  15.52  9.47  0.24  2.96  6.49  4.06   

    K2O  P2O5   H2O  CO2ppm   STppm  Xppm  Fe3FeT  
0  0.38  0.22  1.88    13.0  362.83   0.0   0.171  


For this example we will use the default options in VolFe, which can be found below:

In [3]:
# print default options in VolFe
print(vf.default_models)

                option
type                  
insolubles         yes
H2S_m              yes
species X           Ar
Hspeciation       none
fO2           Kress91A
...                ...
bulk_O           exc_S
error              0.1
print status       yes
output csv         yes
setup               no

[62 rows x 1 columns]


And below runs the calculation - it outputs the index of the sample, the sample name and Pvsat plus a dataframe of the full results.

In [4]:
# runs the calculation
vf.calc_Pvsat(my_analysis)

0 TN273-01D-01-01 337.80934489141146


,sample,T_C,P_bar,SiO2_wtpc,TiO2_wtpc,Al2O3_wtpc,FeOT_wtpc,MnO_wtpc,MgO_wtpc,CaO_wtpc,...,KHOSg opt,KOSg opt,KOSg2 opt,KCOg opt,KCOHg opt,KOCSg opt,KCOs opt,carbonylsulfide opt,density opt,Date
0,TN273-01D-01-01,1200.0,337.809345,57.03956,1.661735,15.536223,9.479899,0.240251,2.963094,6.496784,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-04-04 19:38:34.710754


Alternatively, we can define the oxygen fugacity using DFMQ and total iron from Fe2O3T...

In [5]:
# Define the melt composition, fO2 estimate, and T as a dictionary.
my_analysis = {'Sample':'TN273-01D-01-01',
           'T_C': 1200.,
           'SiO2': 56.98,
           'TiO2': 1.66,
           'Al2O3': 15.52,
           'Fe2O3T': 9.47, # Fe2O3T instead of FeOT
           'MnO': 0.24,
           'MgO': 2.96,
           'CaO': 6.49,
           'Na2O': 4.06,
           'K2O': 0.38,
           'P2O5': 0.22,
           'H2O': 1.88,
           'CO2ppm': 13.,
           'STppm': 362.83,
           'Xppm': 0.,
           'DFMQ': 0.} # DFMQ instead of Fe3+/FeT

# Turn the dictionary into a pandas dataframe, setting the index to 0.
my_analysis = pd.DataFrame(my_analysis, index=[0])

# runs the calculation
vf.calc_Pvsat(my_analysis)

0 TN273-01D-01-01 317.21803054879365


,sample,T_C,P_bar,SiO2_wtpc,TiO2_wtpc,Al2O3_wtpc,FeOT_wtpc,MnO_wtpc,MgO_wtpc,CaO_wtpc,...,KHOSg opt,KOSg opt,KOSg2 opt,KCOg opt,KCOHg opt,KOCSg opt,KCOs opt,carbonylsulfide opt,density opt,Date
0,TN273-01D-01-01,1200.0,317.218031,60.242534,1.755047,16.408637,4.504553,0.253742,3.129482,6.861601,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-04-04 19:38:35.245028


... or FeO and Fe2O3 ...

In [6]:
# Define the melt composition, fO2 estimate, and T as a dictionary.
my_analysis = {'Sample':'TN273-01D-01-01',
           'T_C': 1200.,
           'SiO2': 56.98,
           'TiO2': 1.66,
           'Al2O3': 15.52,
           'FeO': 8.03, # FeO and Fe2O3 to contraint total Fe and fO2
           'Fe2O3': 2.57, # ^
           'MnO': 0.24,
           'MgO': 2.96,
           'CaO': 6.49,
           'Na2O': 4.06,
           'K2O': 0.38,
           'P2O5': 0.22,
           'H2O': 1.88,
           'CO2ppm': 13.,
           'STppm': 362.83,
           'Xppm': 0.} 

# Turn the dictionary into a pandas dataframe, setting the index to 0.
my_analysis = pd.DataFrame(my_analysis, index=[0])

# runs the calculation
vf.calc_Pvsat(my_analysis)

0 TN273-01D-01-01 436.42964854065804


,sample,T_C,P_bar,SiO2_wtpc,TiO2_wtpc,Al2O3_wtpc,FeOT_wtpc,MnO_wtpc,MgO_wtpc,CaO_wtpc,...,KHOSg opt,KOSg opt,KOSg2 opt,KCOg opt,KCOHg opt,KOCSg opt,KCOs opt,carbonylsulfide opt,density opt,Date
0,TN273-01D-01-01,1200.0,436.429649,57.205223,1.666561,15.581345,9.222566,0.240949,2.9717,6.515653,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-04-04 19:38:35.745185


... DNNO and FeOT ...

In [7]:
# Define the melt composition, fO2 estimate, and T as a dictionary.
my_analysis = {'Sample':'TN273-01D-01-01',
           'T_C': 1200.,
           'SiO2': 56.98,
           'TiO2': 1.66,
           'Al2O3': 15.52,
           'FeOT': 9.47, ###
           'MnO': 0.24,
           'MgO': 2.96,
           'CaO': 6.49,
           'Na2O': 4.06,
           'K2O': 0.38,
           'P2O5': 0.22,
           'H2O': 1.88,
           'CO2ppm': 13.,
           'STppm': 362.83,
           'Xppm': 0.,
           'DNNO': 1.} ###

# Turn the dictionary into a pandas dataframe, setting the index to 0.
my_analysis = pd.DataFrame(my_analysis, index=[0])

# runs the calculation
vf.calc_Pvsat(my_analysis)

0 TN273-01D-01-01 451.9452953846262


,sample,T_C,P_bar,SiO2_wtpc,TiO2_wtpc,Al2O3_wtpc,FeOT_wtpc,MnO_wtpc,MgO_wtpc,CaO_wtpc,...,KHOSg opt,KOSg opt,KOSg2 opt,KCOg opt,KCOHg opt,KOCSg opt,KCOs opt,carbonylsulfide opt,density opt,Date
0,TN273-01D-01-01,1200.0,451.945295,57.03956,1.661735,15.536223,9.479899,0.240251,2.963094,6.496784,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-04-04 19:38:36.233896


If we try to specify too many variables to constrain total Fe and fO2, we'll get a warning from VolFe telling you it is only using one of the options to calculate things.

In [8]:
# Define the melt composition, fO2 estimate, and T as a dictionary.
my_analysis = {'Sample':'TN273-01D-01-01',
           'T_C': 1200.,
           'SiO2': 56.98,
           'TiO2': 1.66,
           'Al2O3': 15.52,
           'FeOT': 9.47, ###
           'MnO': 0.24,
           'MgO': 2.96,
           'CaO': 6.49,
           'Na2O': 4.06,
           'K2O': 0.38,
           'P2O5': 0.22,
           'H2O': 1.88,
           'CO2ppm': 13.,
           'STppm': 362.83,
           'Xppm': 0.,
           'DNNO': 1.,
           'Fe3FeT':0.171} ###

# Turn the dictionary into a pandas dataframe, setting the index to 0.
my_analysis = pd.DataFrame(my_analysis, index=[0])

# runs the calculation
vf.calc_Pvsat(my_analysis)

/Users/ehughes/Documents/GitHub/VolFe/src/VolFe/melt_gas.py:442: UserWarning: you entered more than one way to infer iron speciation, note that this calcualtion is only considering the entered Fe3+/FeT
  w.warn('you entered more than one way to infer iron speciation, note that this calcualtion is only considering the entered Fe3+/FeT')


0 TN273-01D-01-01 337.80934489141146


,sample,T_C,P_bar,SiO2_wtpc,TiO2_wtpc,Al2O3_wtpc,FeOT_wtpc,MnO_wtpc,MgO_wtpc,CaO_wtpc,...,KHOSg opt,KOSg opt,KOSg2 opt,KCOg opt,KCOHg opt,KOCSg opt,KCOs opt,carbonylsulfide opt,density opt,Date
0,TN273-01D-01-01,1200.0,337.809345,57.03956,1.661735,15.536223,9.479899,0.240251,2.963094,6.496784,...,KO97,KO97,OM22,KO97,KO97,Moussallam19,Holloway92,COS,DensityX,2024-04-04 19:38:36.753740
